<a href="https://colab.research.google.com/github/JAPJ182/BANREP/blob/main/BANREP/Scripts/Python/Scrapping_Tutelas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step 1. Active all libraries incluid the function for scrapping tutelas in Colombia

In [ ]:
!pip install html-table-parser-python3
url =  "https://raw.githubusercontent.com/JAPJ182/BANREP/main/Scripts/scrapping_tutelas.py"
!wget --no-cache --backups=1 {url}

exec(compile(open("scrapping_tutelas.py", "rb").read(), "scrapping_tutelas.py", 'exec'))
import urllib.request
from pprint import pprint
from html_table_parser.parser import HTMLTableParser
import pandas as pd
import requests
import lxml.html as lh
import time
!rm scrapping_tutelas.py
from google.colab import drive
drive.mount('drive')
folder_drive = "/content/drive/MyDrive/Tutelas EPS/"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-12-21 15:39:19--  https://raw.githubusercontent.com/JAPJ182/BANREP/main/Scripts/scrapping_tutelas.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2021 (2.0K) [text/plain]
Failed to rename scrapping_tutelas.py to scrapping_tutelas.py.1: (2) No such file or directory
Saving to: ‘scrapping_tutelas.py’

scrapping_tutelas.p 100%[===================>]   1.97K  --.-KB/s    in 0s      

2022-12-21 15:39:20 (37.3 MB/s) - ‘scrapping_tutelas.py’ saved [2021/2021]

Mounted at drive


# Step 2. A loop is created to extract each possible Guardianship defendant agreement for each month, EPS and year.

In [ ]:
anio = [ '2000' , '2001' , '2002' , '2003' , '2004' , '2005' , 
        '2006' , '2007' , '2008' , '2009' , '2010' , '2011' ,
        '2012' , '2013' , '2014' , '2015' , '2016' , '2017' ,
        '2018' , '2019' , '2020' , '2021', '2022' ]
mes = ['01' , '02' , '03'  , '04'  , '05'  , '06' , '07'  , '08', '09' , '10' , '11' , '12' ]
keyword = [
   'eps', 'e.p.s' ,	
   'CAJA DE COMPENSACION FAMILIAR DEL HUILA', 'COMFAMILIAR HUILA',
     'CAMACOL'  , 'COMFAMA' , 'COMFAMILIAR CARTAGENA' , 'COMFABOY' , 'COLSUBSIDIO' ,	'FAMISANAR' ,	'CAFAM' ,
    'COMFACOR' ,	'CAJA DE COMPENSACION FAMILIAR DE CORDOBA' ,	 'COMFAGUAJIRA' ,	'CAJA DE COMPENSACION FAMILIAR DE LA GUAJIRA' ,	'COMFAMILIAR GUAJIRA' ,
       'COMFAMILIAR NARIÑO' ,	'CAJA DE COMPENSACION FAMILIAR DE NARIÑO' ,	
     'COMFAMILIAR NARIÑO' ,	'COMFENALCO QUINDIO' ,	'CAJA DE COMPENSACION FAMILIAR DE FENALCO' , 'COMFAMILIAR RISARALDA' ,		
     'CAJASAN' ,	'CAJA SANTANDEREANA DE SUBSIDIO FAMILIAR' ,	'COMFENALCO SANTANDER' , 'COMFASUCRE' , 'CAFABA' ,	'CAJA DE COMPENSACION FAMILIAR DE BARRANCABERMEJA' ,	
     'COMFENALCO TOLIMA' ,		'COMFACARTAGO' ,		'COMFANORTE' ,	'CAJA DE COMPENSACION FAMILIAR DEL NORTE DE SANTANDER' ,	
     'COMFAORIENTE' ,	'CAJA DE COMPENSACION FAMILIAR CCF DEL ORIENTE COLOMBIANO ' ,	'COMFACUNDI' ,	'CAJA DE COMPENSACION FAMILIAR DE CUNDINAMARCA' ,	
     'COMFENALCO CUNDINAMARCA' ,		'CAJACOPI ATLANTICO' , 'COMFACHOCO' , 'COMFACA' , 'EMPRESAS PUBLICAS DE MEDELLIN-DEPARTAMENTO MEDICO' ,		
      'FERROCARRILES' ,		'ALIANSALUD ' ,
       'SALUD TOTAL ' , 'CAFESALUD' , 'SANITAS' ,		'SEGUROS SOCIALES ' , 'COMPENSAR' ,		
      'COMFENALCO ANTIOQUIA' ,		'SURAMERICANA' , 'COMFENALCO VALLE' ,
         'SALUDCOOP' ,		'HUMANA VIVIR' , 'COLPATRIA' ,		
      'COOMEVA' ,		'SERVICIO OCCIDENTAL DE SALUD' , 'CAPRECOM' , 'CONVIDA' ,
      'CRUZ BLANCA' , 'CAPRESOCA' ,		
      'SOLSALUD' ,		'CALISALUD' , 'CONDOR  SA' , 'SELVASALUD' , 'SALUDVIDA' , 'SALUDCOLOMBIA' , 'SALUD ATENCION HUMANA' ,		
      'NUEVA EPS' ,		'MULTIMEDICAS SALUD' , 'GOLDEN GROUP' , 'COOSALUD' ,
       'MEDIMAS' , 'SALUD MIA' , 'MUTUAL SER' ,		
       'DUSAKAWI' ,		'MANEXKA' , 'ASOCIACION INDIGENA DEL CAUCA' , 'ANASWAYUU' , 'MALLAMAS' , 'PIJAOS SALUD' ,		
       'CAPITAL SALUD' ,		'SAVIA SALUD' , 'EMDISALUD ' ,
       'COOSALUD' , 'ASMET  SALUD' ,	'ASMET  ' , 'ASTREA' ,		
      'AMBUQ' ,	'ASOCIACION MUTUAL BARRIOS UNIDOS DE QUIBDO' ,
      	'ECOOPSOS' ,	'ENTIDAD COOPERATIVA SOLDE SALUD DEL NORTE DE SOACHA' ,	
      'MALLAMAS' ,	'ASOCIACION MUTUAL' ,	'EMSSANAR' ,	'ASOCIACION MUTUAL EMPRESA SOLIDARIA DE SALUD DE NARIÑO ' ,	
      'COMPARTA' ,	'COOPERATIVA DE SALUD COMUNITARIA' , 'ASOCIACION MUTUAL SER EMPRESA SOLIDARIA DE SALUD' ,		
      'FUERZAS MILITARES' ,		'POLICIA NACIONAL' , 'ECOPETROL' , 'MAGISTERIO' ,		
      'UATLANTICO' ,		'INPEC'  , 'CAJASALUD' ,		
      'COMFAMILIARES EN SALUD UT' , 'CONVENIO COMFENALCO UT' , 'CONVENIO CAMACOL CONFAMA UT' ,
      'empresa prestadora de salud'

]

# 
for i in keyword:
  # primero: descarga todos lso registros, segun la palabra clave
  time.sleep(5)

  try:
    appended_data = []
    temp = scrap_tutelas( keyword = i )
  except IndexError as e:
    print(f"{e}")
  except UnicodeEncodeError as U:
    print(f"{U}")
  #si esta priemra descarga tiene menos de 5500 registros entonces de inmediato se imprime el csv del archivo. 
  if temp.shape[0] <= 5500:
    print('se corre la eps {}'.format(i))
    temp['keyword'] = i
    temp.to_parquet(folder_drive+i+'.parquet',index=False  )
    # si existen mas de 5500 registros entonces descargamos por cada año, en este casod esde el 2000 al 2022
  else:
    for j in anio:
      # 
        try:
          
          temp = scrap_tutelas( keyword = i, anio = j) 
          if temp.shape[0] >= 5500:
    #Ahora bien si al descargar por año tambien tiene mas de 5500 entonces se descarga por ems
            for k in mes:
              print('se corre la eps {}, del año {}, y el mes {} '.format(i,j, k )) #,k 
              temp = scrap_tutelas( keyword = i , anio = j, mes= k ) # 
              appended_data.append(temp)
          else:
            print('se corre la eps {}, del año {} '.format(i,j)) #,k , y el mes {}
            appended_data.append(temp)

          df = pd.concat(appended_data)
          df['keyword'] = i
          df.to_parquet(folder_drive+i+'.parquet', index=False )

        except IndexError as e:
          print(f"{e}")
          pass



se corre la eps empresa prestadora de salud
